# Coursera -- IBM Applied Data Science Capstone Project Week 3


This notebook will be mainly used for IBM course applied data science capstone week 1 project

<h1>Table of contents</h1>

1.Scrape the Toronto postal code dataframe from Wikipedia page. The dataframe should have three columns:'Postal Code','Borough' and       'Neighbourhood'. 
  * Transform the table into a _pandas_ dataframe.
  * Ignore cells with a borough that is Not assigned.
  * For rows that have different neighbourhood names but have the same postal code, combine them into one row with the neighborhoods separated with a comma.
  * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

2.Get the latitude and the longitude coordinates of each neighborhood, add 'Latitude' and 'Longitude' columns to the Toronto postal code dataframe.Visualize the data.

3.Explore and cluster the neighborhoods in Toronto. I will work with only boroughs that contain the word 'Toronto '.

## 1. Scrape the Toronto postal code dataframe.

In [1]:
import pandas as pd
from pandas import DataFrame

In [2]:
df_list = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)

df_list is a list type data structure which contains all the tables from 'List of postal codes of Canada: M' webpage. We will only need the first table, which is df_list[0]

In [3]:
df = df_list[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
df = df.loc[~(df['Borough'] == 'Not assigned')].reset_index(drop=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Check if there is more than one neighbourhood existing in one postal code area.

In [ ]:
len(df['Postal Code'].unique())

103

There is 103 unique postal codes and the df dataframe has 103 rows. So there is no duplicate postal code existing in our dataframe. Next, let's check if there is 'not assigned' neighbourhood.

In [ ]:
df.loc[(df['Neighbourhood'] == 'Not assigned')].shape


(0, 3)

There is no 'not assigned' neighbourhood. The shape of our cleaned dataframe is

In [ ]:
df.shape

(103, 3)

In [ ]:
print('The dataframe has {} boroughs'.format(
        len(df['Borough'].unique())
    )
)

The dataframe has 10 boroughs


## 2. Get coordinates for each neighbourhood

In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighbourhood.There are two ways to get the geographical coordinates of the neighbourhoods:

1. Using the Geocoder Python package
2. Using the csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

I will use both methods to get the data

### 1. Using the Geocoder Python package

In [ ]:
!pip install geocoder
print('geocoder installed!')

geocoder installed!


Let's create an empty dataframe called df_cor_geocoder. I will use this new dataframe to store 'Postal Code', 'latitude' and 'longitude' data.

In [ ]:
# define the dataframe columns
column_names = ['Postal Code', 'Latitude', 'Longitude'] 

# instantiate the dataframe
df_cor_geocoder = pd.DataFrame(columns=column_names)


I will run a while loop to get cordinates for each postal code.

In [ ]:
import geocoder # import geocoder

for code in df['Postal Code']:
    # initialize the variable to None
    lat_lng_coords = None
    # loop until I get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        lat_lng_coords = g.latlng

    latitude=lat_lng_coords[0]
    longitude=lat_lng_coords[1]
    df_cor_geocoder = df_cor_geocoder.append({'Postal Code': code,
                                          'Latitude': latitude,
                                          'Longitude': longitude}, ignore_index=True)


In [ ]:
df_cor_geocoder = df_cor_geocoder.sort_values(by=['Postal Code']).reset_index(drop=True)
df_cor_geocoder.head()

In [ ]:
df_cor_geocoder.shape

### 2. Using the csv data
Read the csv data from provided link and store the 'Postal Code', 'Latitude','Longitude' data into a new dataframe called df_cor_csv

In [ ]:
df_cor_csv = pd.read_csv('http://cocl.us/Geospatial_data')
df_cor_csv = df_cor_csv.sort_values(by=['Postal Code'])
df_cor_csv.head()

In [ ]:
df_cor_csv.shape

### 3. Compare two cordinate dataframes

df_cor_geocoder and df_cor_csv have the same shape. They both work for getting codinate data. But I can see slightly different result by checking their first five rows. I can also confirm they are different by using _.equals_ function.

In [ ]:
df_cor_geocoder.equals(df_cor_csv)

The above code returned False, which means the two dataframes have different data. Now I will check how big the difference is. First, Let me create a new dataframe call df_cor_diff. I will caculate the difference of latitude and longtitude data for each postal code and store the difference data in the new dataframe.

In [ ]:
df_cor_diff = pd.DataFrame(columns=['Postal Code','Latitude_diff','Longitude_diff'])
df_cor_diff.head()

In [ ]:
for i in range(0,df_cor_csv.shape[0]):
    df_cor_diff = df_cor_diff.append({'Postal Code': df_cor_csv['Postal Code'][i],
                                    'Latitude_diff': round((df_cor_geocoder['Latitude'][i]-df_cor_csv['Latitude'][i]),3),
                                    'Longitude_diff': round((df_cor_geocoder['Longitude'][i]-df_cor_csv['Longitude'][i]),3)}, ignore_index=True)

df_cor_diff.head()

In [ ]:
print('The biggest latitude difference is {}, and the biggest longitude difference is {}\n'
      .format(abs(df_cor_diff['Latitude_diff'].max()), abs(df_cor_diff['Longitude_diff'].max())))
print('The row with biggest latitude difference:')
print(df_cor_diff.loc[abs(df_cor_diff['Latitude_diff'])==0.012])
print('\n')

print('The row with biggest longitude difference:')
print(df_cor_diff.loc[abs(df_cor_diff['Longitude_diff'])==0.23])
print('\n')

The biggest longitude differnce is large compared to latitude difference. More specifically, the two data sets give both biggest  logitude difference and latitude difference for postal code 'M7R'. Let's look more closely. 

In [ ]:
print('Data from geocoder method for postal code \'M7R\'')
print(df_cor_geocoder.loc[df_cor_geocoder['Postal Code']=='M7R'])
print('\n')
print('Data from csv file for postal code \'M7R\'')
print(df_cor_csv.loc[df_cor_csv['Postal Code']=='M7R'])

By using an [online Latitude/Longtidue Calculator](https://www.nhc.noaa.gov/gccalc.shtml), I know that the distance between (43.64869  -79.38544) and (43.636966 -79.615819) is around 19 km. That is pretty large. For example, if we use geocoder data to calculate the distance between two postal codes, let's say,'M1B' and "M1C', the distance is only about 4 km. However, although the two different cordinate dataset are not 100% same, it won't affect how I visualize and cluster the data. Thus I will go ahead using geocoder dataframe to merge with previous df dataframe to create a new dataframe that has all the information needed for the map. 

In [ ]:
# substitute df_cor_geocoder with df_cor_csv if you want to use csv data 
df_cor = df_cor_geocoder

### 4.Data Visualization

Merge df and df_cor as a new dataframe 

In [ ]:
df_all = pd.merge(df,df_cor,on ='Postal Code')
df_all

In [ ]:
!pip install "folium==0.11.0"
import folium

print('Folium installed and imported!')

In [ ]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [ ]:
address = 'Toronto,Canada'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_all['Latitude'], df_all['Longitude'], df_all['Borough'], df_all['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 3.Explore and cluster the neighborhoods in Toronto
Let's simplify the above map and segment and cluster only boroughs that contain the word Toronto.So let's slice the original dataframe and create a new dataframe of the data that has required boroughs name.

In [ ]:
name_data = df_all[df_all['Borough'].str.contains('Toronto')].reset_index(drop=True)
name_data.head()

In [ ]:
# create map of Borough that contain the world 'Toronto'
map_name = folium.Map(location=[latitude+0.02, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(name_data['Latitude'], name_data['Longitude'], name_data['Borough'], name_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_name)  
    
map_name

Next, we are going to start utilizing the Foursquare API to explore the neighbourhoods and segment them.

In [ ]:
#hidden cell codes 
 
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# The code was removed by Watson Studio for sharing.

Let's create a function to get top 100 venues within a radius of 500 meters of all the neighbourhoods in name_data

In [ ]:
import requests

radius = 500
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
# return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighbourhood and create a new dataframe called *name_venues*.

In [ ]:
name_venues = getNearbyVenues(names=name_data['Neighbourhood'],
                                   latitudes=name_data['Latitude'],
                                   longitudes=name_data['Longitude']
                                  )
name_venues.shape


Let's check the size of the resulting dataframe.

In [ ]:
print(name_venues.shape)
name_venues.head()

Let's check how many venues were returned for each neighbourhood

In [ ]:
name_venues.groupby('Neighbourhood').count()

Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(name_venues['Venue Category'].unique())))

Analyze each neighbourhood

In [ ]:
# one hot encoding
name_onehot = pd.get_dummies(name_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to datafra
name_onehot['Neighbourhood'] = name_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [name_onehot.columns[-1]] + list(name_onehot.columns[:-1])
name_onehot = name_onehot[fixed_columns]

name_onehot.head()

In [ ]:
# new dataframe size
name_onehot.shape

Next, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
name_grouped = name_onehot.groupby('Neighbourhood').mean().reset_index()
name_grouped

In [ ]:
# the grouped new size
name_grouped.shape

In [ ]:
#Let's print each neighbourhood along with the top 5 most common venuesnum_top_venues = 5
num_top_venues = 5

for hood in name_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = name_grouped[name_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Let's put that into a pandas dataframe.
First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
import numpy as np
# print the top 10 venues for each neighbour
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = name_grouped['Neighbourhood']

for ind in np.arange(name_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(name_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()


Cluster Neighbourhoods:
Run k-means to cluster the neighbourhood into 5 clusters.

In [ ]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

name_grouped_clustering = name_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(name_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues.

In [ ]:
print(neighbourhoods_venues_sorted.shape)
print(name_data.shape)

'neighbourhoods_venues_sorted' dataframe contains less rows than 'name_merged' dataframe as there is not enough data available from Foursquare for all the neighbourhoods. So when I merge these two dataframes, the lacking rows from 'neighbourhoods_venues_sorted' (incl. 'Cluster labels' column) are filled with NaN, which converts 'Cluster labels' column into FLOAT. This will cause error because when I visualize the data, the cluster columns will work as indices for color list, which requries them to be integer. To fix this problem, I will change the JOIN() parameter 'how' into 'right'(default is 'left'). This will fix everything as the extra rows from'name_merged' dataframe will be simply ignored upon merging.

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

name_merged = name_data

# merge name_grouped with name_data to add latitude/longitude for each neighbourhood
name_merged = name_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood',how='right')

# check the last columns!

name_merged.head()

Finally let's visualize final cluster results.

In [ ]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(name_merged['Latitude'], name_merged['Longitude'], name_merged['Neighbourhood'], name_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the Clusters.
Now, I can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

### Cluster 1

In [ ]:
name_merged.loc[name_merged['Cluster Labels'] == 0, name_merged.columns[[1] + list(range(5, name_merged.shape[1]))]]

### Cluster 2

In [ ]:
name_merged.loc[name_merged['Cluster Labels'] == 1, name_merged.columns[[1] + list(range(5, name_merged.shape[1]))]]

### Cluster 3

In [ ]:
name_merged.loc[name_merged['Cluster Labels'] == 2, name_merged.columns[[1] + list(range(5, name_merged.shape[1]))]]

### Cluster 4

In [ ]:
name_merged.loc[name_merged['Cluster Labels'] == 3, name_merged.columns[[1] + list(range(5, name_merged.shape[1]))]]

### Cluster 5

In [ ]:
name_merged.loc[name_merged['Cluster Labels'] == 4, name_merged.columns[[1] + list(range(5, name_merged.shape[1]))]]